# Defining Neural Network Architecture


## Architecture :
- 784 neurons (input layer) + 1 bias and relu activation
- 16 neurons + 1 bias and relu activation
- 16 neurons + 1 bias and relu activation
- 10 neurons (output layer) + 1 bias and softmax activation



In [2]:
import numpy as np

In [ ]:
class DNN:
    def __init__(self, learning_rate):
        self.theta_matrix_1 = np.zeros((16, 785))
        self.theta_matrix_2 = np.zeros((16, 17))
        self.theta_matrix_3 = np.zeros((10, 17))
    
    def relu(self, inp:np.array):
        a = []
        for i in inp:
            if(i > 0):
                a.append(i)
            else:
                a.append(np.array([0]))
        a = np.array(a)
        return(a)
    
    def softmax(self, inp:np.array):
        a = []
        denominator = np.sum(np.exp(inp))
        for i in inp:
            numerator = np.exp(i)
            probability = numerator/denominator
            a.append(probability)
        a = np.array(a)
        return(a)
    
    def forwardpropogation(self, x:np.array):
        x = np.reshape(x, (x.shape[0], 1))
        z1 = x
        a1 = self.relu(z1)
        a1 = np.insert(a1, 0, 1, axis=0)
        
